In [74]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [75]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from common.extractor.alpaca_extractor import AlpacaExtractor
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio import Portfolio
import numpy as np
import matplotlib.pyplot as plt

In [76]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [77]:
orivault = ADatabase("ori")
alp = AlpacaExtractor(paper=False)

In [78]:
orivault.connect()
portfolio_dictionary = orivault.retrieve("results").to_dict("records")[0]
orivault.disconnect()
portfolio_dictionary

{'ranking_metric': 'roc',
 'timeframe': 'DAY',
 'position_type': 'long',
 'grouping_type': 'STANDARD_DEV',
 'selection_type': 'bottom_blacklist',
 'allocation_type': 'risk',
 'risk_type': 'coefficient_of_variation',
 'selection_percentage': 1,
 'stoploss': 1,
 'num_of_groups': 1,
 'max_price': 0.1,
 'min_price': 0.01,
 'max_market_cap': 1000000,
 'min_market_cap': 10000,
 'rolling_window': 10,
 'leverage': 1,
 'date': Timestamp('2025-03-14 00:00:00'),
 'pnl': 6.695519929884597,
 'raw_pnl': 10.373208896269675,
 'downside': 1.4242520632259887,
 'coefficient_of_variation': 3.1651236135779435,
 'portfolio_std': 2.115405509333582,
 'sharpe_ratio': 2.6919501734589018,
 'tracking_error': 2.1151877769808967,
 'information_ratio': 2.6922272763482797}

In [79]:
pm = Portfolio.from_dict(portfolio_dictionary)
pm.to_dict()

{'ranking_metric': 'roc',
 'timeframe': 'DAY',
 'position_type': 'long',
 'grouping_type': 'STANDARD_DEV',
 'selection_type': 'bottom_blacklist',
 'allocation_type': 'risk',
 'risk_type': 'coefficient_of_variation',
 'selection_percentage': 1.0,
 'stoploss': 1.0,
 'num_of_groups': 1,
 'max_price': 0.1,
 'min_price': 0.01,
 'max_market_cap': 1000000.0,
 'min_market_cap': 10000.0,
 'rolling_window': 10,
 'leverage': 1}

In [80]:
## Retrieve existing stocks 
index = alp.assets().sort_values("ticker")
tickers = index["ticker"].values
tickers_per_batch = int(1000)
batchs = [tickers[i:i + tickers_per_batch] for i in range(0, len(tickers), tickers_per_batch)]
bars = []
[bars.append(alp.latest_bars_bulk(batch)) for batch in batchs]
todays_bar = pd.concat(bars)
index = index.merge(todays_bar,on="ticker",how="left")

'SVA'


In [81]:
end = alp.clock() - timedelta(days=1)
start = (end - timedelta(days=pm.rolling_window*3))
print(start,end)

2025-04-25 07:52:13.148982-04:00 2025-05-25 07:52:13.148982-04:00


In [82]:
tickers_per_batch = int(10000/(pm.rolling_window*4)/5)
print(tickers_per_batch)

50


In [83]:
relevant_tickers = index[(index["adjclose"]>=pm.min_price) & (index["adjclose"]<=pm.max_price)]["ticker"]
batchs = [relevant_tickers[i:i + tickers_per_batch] for i in range(0, len(relevant_tickers), tickers_per_batch)]

In [84]:
prices = []
for batch in tqdm(batchs):
    try:
        tickers_data = alp.prices_bulk(batch,start,end)
        for ticker in batch:
            try:
                price = tickers_data[tickers_data["ticker"] == ticker].copy()
                bar = todays_bar[todays_bar["ticker"]==ticker][["date","ticker","adjclose","high","low","volume"]].copy()
                price = p.lower_column(price)
                price = pd.concat([price,bar])
                price = p.utc_date(price)
                price.sort_values("date", inplace=True)
                price = p.additional_date_columns(price)
                price["market_cap"] = price["adjclose"] * price["volume"]
                if price[(price["adjclose"]<=pm.max_price) & (price["adjclose"]>=pm.min_price)].index.size > 0:
                    price["standard_dev"] = 1
                    price["coefficient_of_variation"] = 1
                    price = Indicator.indicator_type_factory(pm.ranking_metric).calculate(price,timeframe=pm.rolling_window,live=True)
                    prices.append(price)
            except Exception as e:
                print(str(e))
    except Exception as e:
        print(str(e))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.36it/s]


In [85]:
simulation = pd.concat(prices)
simulation.sort_values("date", inplace=True)
trades = pm.recs(simulation.copy())
trades

,year,day,ticker,date,adjclose,high,low,volume,month,quarter,...,weekday,market_cap,standard_dev,risk,roc,group_percentile,rank_percentile,major_key,position_type,weight
19,2025,143,LYRA,2025-05-23 00:00:00+00:00,0.0918,0.0967,0.088,999767,5,2,...,4,91778.6106,1,1,-0.065173,1,815,2025_143_1,1.0,1.0


In [86]:
options = alp.call_options("GDHG",datetime(2025,5,23))
options

dict_keys([])

In [87]:
orivault.connect()
orivault.drop("recommendations")
orivault.store("recommendations",trades)
orivault.disconnect()